In [ ]:

import math
import numpy as np
import scipy 
import scipy.stats as sts
import matplotlib
import matplotlib.pyplot as plt
%matplotlib inline
s_vi=25
s_ve=35
g_vi=1*80
g_ve=1*150
tuning_curve_disc=np.linspace(0,180,30)

# Tutorial 1

## Introduction

In the first tutorial of Module 3, we will implement a simple probabilistic population code, performing Bayes-optimal multisensory integration given some simple assumptions. We will encode the heading direction of a subject in two neural populations with Poisson variability, then integrate the information from these populations to calculate the posterior through a simple application of Bayes' rule.


## Exercise 1
First we will simulate population responses representing heading direction in the visual and vestibular system. We will assume Bell-shaped (Gaussian) tuning curves, with different gains for each population, and Poisson variability.

Write a function to generate, given an angle, the response rates in the visual and vestibular populations, with 30 neurons each. Neurons in each population should have equally spaced preferred stimuli between 0 and 180 degrees, and otherwise identical tuning curves $f_i(s)$. You are welcome to experiment with different values for the gains and for the standard deviation that defines the shape of the tuning curves, or you can use the following:  $g_{vi}=80$, $g_{ve}=150$, and $\sigma=40$.

Then write a function that generates spike counts for these populations, using independent Poisson variability.

$\displaystyle p(\mathbf r \rvert s)=\prod_{i=1}^{N} p(r_i \rvert s)=\prod_{i=1}^{N}\frac{e^{-gf_{i}(s)}g \cdot f_i(s)}{r_i!}$

Plot these spike counts for the two populations for an input of 80$^{\circ}$.

<img src="spikes_11.png">


In [ ]:
def rates(s):
    vi_rates=g_vi*sts.norm.pdf(s,tuning_curve_disc,s_vi)
    ve_rates=g_ve*sts.norm.pdf(s,tuning_curve_disc,s_ve)
    
    return vi_rates, ve_rates,


def counts(s):
    
    vi_rates,ve_rates=rates(s)  
    vi_sp=np.random.poisson(vi_rates)
    ve_sp=np.random.poisson(ve_rates)
    return vi_sp,ve_sp

r_counts=counts(80)
visual=plt.plot(tuning_curve_disc,r_counts[0],'o',label='visual')
vestibular=plt.plot(tuning_curve_disc,r_counts[1],'o',label='vestibular')
combined=plt.plot(tuning_curve_disc,r_counts[1]+r_counts[0],'o',label='combined')
plt.ylabel('# of spikes')
plt.xlabel('Preferred angle for neuron')
plt.yticks(range(max(r_counts[1]+r_counts[0])+2))
plt.legend()
plt.show()


## Exercise 2
Write a function to calculate the (discretized) posterior over possible heading directions: 
\begin{eqnarray}
p(s\rvert \mathbf r) \propto exp\Big( \sum_{i}\big( -g \cdot f_i(s)+r_i \cdot \log (g\cdot f_i(s)\big) \Big),
\end{eqnarray}
separately for each of our neural populations. Then perform optimal cue integration by adding up the activity from the two populations $\mathbf r_{com} =\mathbf r_{ve}+\mathbf r_{vi}$, and use it to calculate the joint posterior $p(s\rvert \mathbf r_{ve},\mathbf r_{vi})$. To derive the formula for this posterior, simply note that because of conditional independence given s, $p(s\rvert \mathbf r_{ve},\mathbf r_{vi})=p(s\rvert \mathbf r_{vi})p(s\rvert \mathbf r_{ve})$

Use a discretization of at least 100 points for the range of 0$^{\circ}$ to 180$^{\circ}$ of potential heading directions.

Plot the resulting posteriors. Also estimate (numerically) the variance of each posterior distribution, and confirm the analytical relationship $\frac{1}{\sigma_{com}^2}=\frac{1}{\sigma_{ve}^2}+\frac{1}{\sigma_{vi}^2}$.

<img src="posteriors_2.png">

In [ ]:
def get_posterior(s_input, trial):
    r_counts=counts(s_input)
   
    post_len=180
    poterior=np.zeros(post_len)
    ss=np.linspace(0,180,post_len)
    log_posterior_com=np.zeros(post_len)
    log_posterior_ve=np.zeros(post_len)
    log_posterior_vi=np.zeros(post_len)
    for sc in xrange(post_len):
        s=ss[sc]
        
        log_posterior_com[sc]=np.sum(-(rates(s)[0]+rates(s)[1])+r_counts[0]*np.log(rates(s)[0])+r_counts[1]*np.log(rates(s)[1]))
        log_posterior_vi[sc]=np.sum(-rates(s)[0]+r_counts[0]*np.log(rates(s)[0]))
        log_posterior_ve[sc]=np.sum(-rates(s)[1]+r_counts[1]*np.log(rates(s)[1]))
    
    a= [log_posterior_com,log_posterior_ve,log_posterior_vi]
    
    for kc in xrange(len(a)):
        k=a[kc]
        k=np.exp(k)/sum(np.exp(k))
        a[kc]=k
    if trial%100==0:
       
        plt.plot(a[2],'o',label='visual')
        plt.plot(a[1],'o',label='vestibular')
        plt.plot(a[0],'o',label='combined')
        plt.ylabel('posterior density')
        plt.xlabel('Heading direction')
        
        plt.legend()
        plt.show()
      
    errors=[]
    errors2=[]
    estimates=[]
    for k in enumerate(a):
        
        
        estimate=np.dot(ss,k[1])
        error=np.linalg.norm(estimate-s_input)
        errors.append(error)
        
        
        estimate2=ss[np.argmax(k[1])]
        error2=np.linalg.norm(estimate2-s_input)
        errors2.append(error2)
       
        estimates.append(estimate2)
    var=np.zeros(3)
    var[0]=np.dot(a[0],np.arange(180)**2)-(np.dot(a[0],np.arange(180)))**2
    var[1]=np.dot(a[1],np.arange(180)**2)-(np.dot(a[1],np.arange(180)))**2
    var[2]=np.dot(a[2],np.arange(180)**2)-(np.dot(a[2],np.arange(180)))**2
    
    check_equation=np.dot(np.array([-1,1,1]),var**-1)
    
    
    return errors, errors2, estimates
s_input=180*np.random.rand()
trial=0
get_posterior(s_input, trial)

Your function should also return an estimate of the angle, such as the posterior mean or the maximum a posteriori (MAP) estimate (these are equal for a Gaussian posterior, but may differ numerically-which one do you think is better in this case?) for each population given an input, and the corresponding estimation error. Run 100 trials with random input angles and compare the total errors when using the separate and combined populations.

Then change your code s.t. the same stimulus is presented repeatedly for a hundred trials, and estimate the variance of your stimulus estimate for each population. Confirm that the same relationship holds for the trial to trial fluctuations of the estimate as for the variances of the single-trial posterior.   

In [ ]:
cum_errors=np.zeros(3)
cum_errors2=np.zeros(3)
for trial in xrange(100):
    s_input=180*np.random.rand()
    errors, errors2,_=get_posterior(s_input, trial)
    cum_errors+=errors
    cum_errors2+=errors2

print cum_errors
print cum_errors2



In [ ]:
estimates=[]
s_input=180*np.random.rand()
for trial in xrange(100):
   
    _, _,estimate=get_posterior(s_input, trial)
    
    estimates.append(estimate)
    
variances=np.var(np.asarray(estimates),axis=0)
print np.dot(variances**-1,np.array([-1,1,1]))

## Exercise 3

In real brains, variability is often neither exactly Poisson, nor is it independent between neurons, and tuning curves might not be matched exactly between brain regions and neurons. However, a simple linear scheme for representing a joint posterior without loss of information can still be implemented, if certain conditions on the tuning curves and the noise covariance are met. 

In particular we will consider the case of Poisson like variability, where the noise is from the exponential family with linear sufficient statistics:

$p(\mathbf r\rvert s)=\frac{\Psi(\mathbf r)}{\eta (s)}e^{\mathbf h(s) \cdot \mathbf r}$. (Here $\eta (s)$ serves to normalize the distribution.)

Then it can be shown, that in this case
$\mathbf h^{'}(s)=\Sigma^{-1}(s)\cdot\mathbf f^{'}(s)$

In our first example we had $h_i(s)=\log f_i(s)$, with identical tuning curves for our two input populations, and a diagonal covariance matrix. More generally, the necessary conditions will still be met if the tuning curves can be linearly mapped on to a common basis of tuning functions, s.t.
$\mathbf h_{vi}(s)=\mathbf W_{vi}\mathbf H(s)$ and $\mathbf h_{ve}(s)=\mathbf W_{ve}\mathbf H(s)$. 

Then the linear combination $\mathbf r_{com} = \mathbf{W_{vi}^{T}r_{vi}+W_{ve}^{T}r_{ve}}$.

Choose a basis H(s) (you could choose e.g. a set of basis functions that is a mix of log Gaussian and log sigmoid functions), and check if you still get a better readout of the combined populations if you rerun the experiments form the previous exercises, using sparse loading weight matrices $\mathbf W$.






